<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/Uwagi_praktyczne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3 .Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---

< [5.3. Techniki szybkiego i stabilnego uczenia GSN.](05_03_FrozenBN_Mish.ipynb) | [5.4. Uwagi praktyczne](Uwagi_praktyczne.ipynb) | [5.5. Procedura trenowania sieci Faster R-CNN z MobileNetV2](05_05_TRAINING.ipynb)  >

# 5.4 Uwagi praktyczne 

##Wersja repozytorium Detectron2
W pracy wykorzystano wersję 0.1.3 repozytorium Detectron2 z 17.05.2020r.<br>
Poniżej podany jest przykład przywrócenia wersji 0.1.3:<br>

    !git clone https://github.com/facebookresearch/detectron2 detectron2_repo
    !cd detectron2_repo && git reset bd2ea47 --hard # v0.1.3
<br>

##JSON

W pracy do zapisu metryk i wyników prac zastosowano format json. W trakcie projektu zidentyfikowano problem z obługą danych typu integer przez moduł Json. Moduł Json obsługuje tylko i wyłączenie dane całkowite z opisem 'int', natomiast w repozytoriarch często są stosowane rozszenia nazw dla danych typu integer.
W celu ominięcia ograniczenia funkcjonalności modułu Json zastosowano funkcję rekukrencyjną konwersji i ujednolicenia nazw dla danych typu integer:

In [ ]:
import json
def to_int(data):
  if isinstance(data,dict):
    for key in data.keys(): data[key]=to_int(data[key])
  if isinstance(data,list):
    for k in range(len(data)): data[k]=to_int(data[k])
  if isinstance(data,tuple):
    data=list(data)
    for k in range(len(data)): data[k]=to_int(data[k]) 
  if 'int' in data.__class__.__name__: 
    data=int(data)
  return data

#json obsluguje calkowite tylko w typie int
with open('Results.json','w') as f:
  json.dump(to_int(results_data),f)

UWAGA. w przypadku kopiowania danych należy zastosować metodę deepcopy(), która kopuję również dane wskazywane w kolejnych warstwach struktury danych.

##DETR

W przypadku repozytorium DETR zastosowanym w Detectron2 w trakcie uczenia z dużymi learning rate (>1e-4) na bazie WIDER FACE, napotkano problem pojawiania się wyników typu np.nan w koordynatach detekowanych obwiedni obiektów.
Dane tego typu charakteryzyją się tym, że nie można wykonać na nich kolejnych obliczeń.
W standarowym repozytorium zastosowano kontrolę przerywającą dalszy proces:<br>
    # degenerate boxes gives inf / nan results
    # so do an early check
    assert (boxes1[:, 2:] >= boxes1[:, :2]).all()
    assert (boxes2[:, 2:] >= boxes2[:, :2]).all()

W celu utrzymania procesu wprowadziliśmy funkcję korekcji danych, w tym podmiany danych dla typu np.nan oraz uporządkowania koordynat.    
Identyfikację wystąpienia wyniku typu nan, wykonano stosując porównanie:<br>
**if x != x == True**<br>
<br>



In [ ]:
class Fix_boxs(torch.nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,boxs): 
    #placing max size in nan value, ordering of boxes coordinates
    #if nan appears put all mask row to True
    self.mask_nan = (boxs!=boxs).sum(axis=-1)[:,None].expand(*boxs.shape) > 0 
    #for nan rows put box max size [0,0,1,1] 
    boxs[self.mask_nan] = torch.tensor([0,0,1,1]*self.mask_nan[:,0].sum(), 
                                       dtype=torch.float, 
                                       device=boxs.device)

    #for coordinates [l,t,r,b] if r < l or b < t put True for r,l and t,b respectively
    self.mask_ord = (boxs[:,2:]<boxs[:,:2]).float()  @ torch.tensor([[1,0,1,0],[0,1,0,1]],dtype=torch.float, device=boxs.device)==1
    #for mask_ord put respectively [r,_,l,_] and [_,b,_,t]
    boxs[self.mask_ord] = boxs[:,[2,3,0,1]][self.mask_ord]
    return boxs

  def backward(self,input):
    input[self.mask_ord] = input[:,[2,3,0,1]][self.mask_ord] 
    #back to input order
    #input[self.mask_nan] = torch.zeros_like(input, device = input.device)[self.mask_nan] 
    #gradient canceling for nan if necessary  
    return input

fix_boxs=Fix_boxs()

UWAGA:<br>
W rozwiązaniu zastosowano klasę torch.nn.Module dla zachowania pełnej funkcjonalności w procesie uczenia. Klasa musi wykonywać odpowiednie korekty w metodzie forward i backward.

Model DETR wymaga repozytorium wersji 0.2 lub starszej.<br>
W pracy wykorzystano wersję 0.2 repozytorium Detectron2<br>

    !cd detectron2_repo && git reset a33fc53 --hard # v0.2

Zgodnie z informacjami z repozytorium na Github autorzy zgłaszają, iż sposób trenowania jest nadal nieprzewidywalny i wymaga dalszych prac.
W przypadku Wider Face, ilość klas ustawiono na 1, a ilość obiektów na 2000.
Trenowanie ponad 200 epok nie przyniosło oczekiwanych resultatów.

##Ekstrakcja cech z wybranych warstw modelu.

W celu ekstrakcji cech z zewnętrznego modelu, zastosowano metodę dokładania warstwy z klasą zapamiętującą dane do późniejszego wykorzystania.
<br>Format opisujący nazwy modułów modelu został zaczerpnięty z funkcji named_modules.

<br>Poniżej przedstawiono przykład dołączenia warstwy do ektrakcji cech modelu.


In [ ]:
from torch import nn

!pip -q install facenet-pytorch
from facenet_pytorch import MTCNN
mtcnn = MTCNN(image_size=224, margin=0, keep_all=True) 

class View(nn.Module):

    def __init__(self):
        super(View, self).__init__()
        self.data = None
        self.gradInput = None

    def forward(self, input):
        self.data = input
        return input

    def backward(self,input, gradOutput):
        self.gradInput = input
        return input

def add_view(net,name, View_Layer, before = False):
  if isinstance(name,str): name=name.split('.')
  if len(name)>1: 
    add_view(net._modules[name[0]],name[1:],View_Layer,before=before)
  else: 
    if before:
      net._modules[name[0]] = nn.Sequential(View_Layer,net._modules[name[0]])
    else:
      net._modules[name[0]] = nn.Sequential(net._modules[name[0]],View_Layer)

for n,m in mtcnn.named_modules():
  print(n,end='; ')
print('')

views ={}
for l in ["onet.softmax6_1","onet.dense6_2","rnet.dense5_2"]:
    print(l,' - Add_View')
    view = View()
    add_view(mtcnn,l,view)
    views[l] = view

; pnet; pnet.conv1; pnet.prelu1; pnet.pool1; pnet.conv2; pnet.prelu2; pnet.conv3; pnet.prelu3; pnet.conv4_1; pnet.softmax4_1; pnet.conv4_2; rnet; rnet.conv1; rnet.prelu1; rnet.pool1; rnet.conv2; rnet.prelu2; rnet.pool2; rnet.conv3; rnet.prelu3; rnet.dense4; rnet.prelu4; rnet.dense5_1; rnet.softmax5_1; rnet.dense5_2; onet; onet.conv1; onet.prelu1; onet.pool1; onet.conv2; onet.prelu2; onet.pool2; onet.conv3; onet.prelu3; onet.pool3; onet.conv4; onet.prelu4; onet.dense5; onet.prelu5; onet.dense6_1; onet.softmax6_1; onet.dense6_2; onet.dense6_3; 
onet.softmax6_1  - Add_View
onet.dense6_2  - Add_View
rnet.dense5_2  - Add_View


In [ ]:
from PIL import Image
img=Image.open('D.jpg')
dboxes,confs = mtcnn.detect(img)


In [ ]:
dboxes

array([[130.05042, 109.73867, 405.13043, 492.75626]], dtype=float32)

In [ ]:
views['onet.softmax6_1'].data, views['onet.dense6_2'].data

(tensor([[1.6094e-03, 9.9839e-01],
         [2.2715e-03, 9.9773e-01],
         [6.4953e-05, 9.9994e-01],
         [5.1406e-04, 9.9949e-01],
         [2.0288e-04, 9.9980e-01],
         [1.0813e-03, 9.9892e-01]]),
 tensor([[ 0.1666, -0.0402, -0.1149, -0.0242],
         [ 0.1598, -0.0361, -0.1217, -0.0219],
         [ 0.1961, -0.0720, -0.0578, -0.0342],
         [ 0.1790, -0.0427, -0.1300, -0.0621],
         [ 0.1450, -0.0807, -0.1443, -0.0803],
         [ 0.1590, -0.0514, -0.0933,  0.0146]]))

In [ ]:
mtcnn

MTCNN(
  (pnet): PNet(
    (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=10)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=16)
    (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (prelu3): PReLU(num_parameters=32)
    (conv4_1): Conv2d(32, 2, kernel_size=(1, 1), stride=(1, 1))
    (softmax4_1): Softmax(dim=1)
    (conv4_2): Conv2d(32, 4, kernel_size=(1, 1), stride=(1, 1))
  )
  (rnet): RNet(
    (conv1): Conv2d(3, 28, kernel_size=(3, 3), stride=(1, 1))
    (prelu1): PReLU(num_parameters=28)
    (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): Conv2d(28, 48, kernel_size=(3, 3), stride=(1, 1))
    (prelu2): PReLU(num_parameters=48)
    (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv3): Conv2d(48, 64,

<br>


< [5.3. Techniki szybkiego i stabilnego uczenia GSN.](05_03_FrozenBN_Mish.ipynb) | [5.4. Uwagi praktyczne](Uwagi_praktyczne.ipynb) | [5.5. Procedura trenowania sieci Faster R-CNN z MobileNetV2](05_05_TRAINING.ipynb)  >

---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3 .Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---